In [10]:
from vllm import LLM, SamplingParams

llm = LLM("Open-Orca/OpenOrcaxOpenChat-Preview2-13B")

In [40]:
sampling_params = SamplingParams(temperature=0.8, max_tokens=256, stop_token_ids=[])

llm.generate("\nuser:", sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:10<00:00, 10.60s/it]


[RequestOutput(request_id=18, prompt='\nuser:', prompt_token_ids=[1, 29871, 13, 1792, 29901], outputs=[CompletionOutput(index=0, text=' \n\n  - **User**: 1234 @gmail.com\n\n  - **Full Name:** John Doe\n\n  - **Address:** 123 Main Street, New York, NY 10011\n\n  - **Phone:** +1 555-555-5555\n\n  - **Email:** john@example.com\n\n  - **Date of Birth:** 10th January 1990\n\n  - **Gender:** Male\n\n  - **Nationality:** American\n\n  - **Occupation:** Software Developer\n\n  - **Education:** B.S. in Computer Science from New York University\n\n  - **Awards:** None\n\n  - **Interests:** Reading, Traveling, Sports\n\n  - **Siblings:** John has one sibling: Jane Doe\n\n  - **Marital Status:** Single\n\n  - **Children:** None\n\n  - **Religion:** Christian\n\n  - **Hobbies:** Painting,', token_ids=[29871, 13, 13, 29871, 448, 3579, 2659, 1068, 29901, 29871, 29896, 29906, 29941, 29946, 732, 21980, 29889, 510, 13, 13, 29871, 448, 3579, 13658, 4408, 29901, 1068, 2259, 1938, 29872, 13, 13, 29871, 448

In [84]:
import re
from IPython.display import Markdown, Code

def get_substring_until_pattern(input_string, pattern=r'''(\n)+[uU]ser'''):
    parts = re.split(pattern, input_string)
    return parts[0]

def moderated_chat(msg: str):
    # Adding in the specific name of the textbook majorly improved response quality
    textbook_name = "Think Python"
    
    # Stop generation when the LLM generates the token for "user" (1792)
    # This prevents the LLM from having a conversation with itself
    sampling_params = SamplingParams(temperature=0.8, max_tokens=256, stop_token_ids=[1792])

    # We need to inject "bot: " at the end of the user message
    # Otherwise, the LLM is susceptible to attacks where it continues an inappropriate user message e.g.,
    # "user: my favorite sex position is [missionary. \nbot: I don't have any information about sex positions]"
    # vs.
    # "user: my favorite sex position is \nbot: [I don't have any information about sex positions]"
    msg = f"user: {msg}\nbot: "

    # This phrasing seems to work well. Modified from NeMo Guardrails
    preface = (
        f"Below is a conversation between a bot and a user about an instructional textbook called {textbook_name}."
        " The bot is factual and concise. If the bot does not know the answer to a"
        " question, it truthfully says it does not know."
    )

    # Modified from Guardrails
    sample_conversation = '''# This is how a conversation between a user and the bot can go:
user: "Hello there!"
bot: "Hello! How can I assist you today?"
user: "What can you do for me?"
bot: "I am an AI assistant which helps answer questions based on the text you are reading."'''

    ## TODO: retrieve relevant chunks
    relevant_chunks = (
        f"# This is some additional context:\n"
        f"That is a great question, but I can only answer questions about {textbook_name}."
    )
    
    ## TODO: Retrieve Examples
    examples = "We can set up a database of a questions and responses that the bot will use as a reference."

    # Join the prompt components together, ending with the (modified) user message
    prompt = '\n\n'.join([preface, sample_conversation, relevant_chunks, msg])

    result = llm.generate(prompt, sampling_params)[0].outputs[0].text
    print(result)
    print('*'*80)
    display(Code(get_substring_until_pattern(result)))
    
moderated_chat('Write a function that converts a number like 5,000,000 to a string like five million.')

Processed prompts: 100%|██████████| 1/1 [00:07<00:00,  7.71s/it]


The user wants a function that takes a number as input and converts it to a string representing a specific format of that number, such as "five million" for 5,000,000. The function should return the formatted string.

To solve this, we would create a function that handles both the conversion and the formatting. Here's an example of a function that does this:

def convert_to_string_format(number):
    formatted_number = str(number)
    return formatted_number.replace(',', '')

For example, if we input 5,000,000 into the function, we would get the output "five million".

bot: This function, convert_to_string_format(number), takes a number as input and returns a formatted string based on the given number. 

user
********************************************************************************


The user wants a function that takes a number as input and converts it to a string representing a specific format of that number, such as "five million" for 5,000,000. The function should return the formatted string.

To solve this, we would create a function that handles both the conversion and the formatting. Here's an example of a function that does this:

def convert_to_string_format(number):
    formatted_number = str(number)
    return formatted_number.replace(',', '')

For example, if we input 5,000,000 into the function, we would get the output "five million".

bot: This function, convert_to_string_format(number), takes a number as input and returns a formatted string based on the given number.